In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import ConfusionMatrixDisplay, accuracy_score
from sklearn.linear_model import Perceptron

study_df = pd.read_csv("Placement_Dataset.csv")
study_df.isna()


candidate_matrix = study_df[["grade_point", "resume_rating"]].values.astype(float)
placed_flag_vec = study_df["placed_flag"].values.astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    candidate_matrix, placed_flag_vec, test_size=0.3, random_state=7, stratify=placed_flag_vec
)

scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

X_train_std[:3], y_train[:3]

perc_sklearn = Perceptron(
    fit_intercept=True,
    max_iter=200,
    tol=None,
    shuffle=True,
    eta0=0.1,
    random_state=7
)

perc_sklearn.fit(X_train_std, y_train)

y_pred_train_sk = perc_sklearn.predict(X_train_std)
y_pred_test_sk  = perc_sklearn.predict(X_test_std)

train_acc_sk = accuracy_score(y_train, y_pred_train_sk)
test_acc_sk  = accuracy_score(y_test,  y_pred_test_sk)

cm_sk = ConfusionMatrixDisplay.from_estimator(
    perc_sklearn,
    X_test_std,
    y_test,
    cmap='plasma'   # 🔹 Changed color scheme
)

print("SKLearn Perceptron")
print("------------------")
print("coef_ (weights):", perc_sklearn.coef_.ravel())
print("intercept_ (bias):", float(perc_sklearn.intercept_))
print(f"Train Accuracy: {train_acc_sk:.3f}")
print(f"Test  Accuracy: {test_acc_sk:.3f}")
print("Confusion Matrix (test):\n", cm_sk)


def step_fn(z):
    return 1 if z >= 0 else 0

def train_perceptron(X, y, eps=0.1, max_passes=100):
    """
    X: (n_samples, n_features) standardized features
    y: (n_samples,) labels in {0,1}
    eps: learning rate
    max_passes: epochs over the dataset
    """
    n_samples, n_features = X.shape
    theta_vec = np.zeros(n_features, dtype=float)  # weights
    offset0 = 0.0                                   # bias

    for epoch in range(max_passes):
        errors = 0
        # Shuffle each epoch for robustness
        idx = np.random.permutation(n_samples)
        for i in idx:
            x_i = X[i]
            y_hat = step_fn(np.dot(theta_vec, x_i) + offset0)
            err = y[i] - y_hat  # in {-1, 0, +1}
            if err != 0:
                theta_vec = theta_vec + eps * err * x_i
                offset0  = offset0  + eps * err
                errors += 1
        # Early stop if perfect pass
        if errors == 0:
            break

    return theta_vec, offset0, epoch + 1

def predict_perceptron(X, theta_vec, offset0):
    logits = X @ theta_vec + offset0
    return (logits >= 0).astype(int)

eps = 0.1
max_passes = 200

theta_vec, offset0, epochs_ran = train_perceptron(X_train_std, y_train, eps=eps, max_passes=max_passes)

y_pred_train = predict_perceptron(X_train_std, theta_vec, offset0)
y_pred_test  = predict_perceptron(X_test_std,  theta_vec, offset0)

train_acc = accuracy_score(y_train, y_pred_train)
test_acc = accuracy_score(y_test, y_pred_test)

cm = ConfusionMatrixDisplay.from_predictions(
    y_test,
    y_pred_test,
    cmap='cividis'   # 🔹 Changed color scheme
)

print(f"Weights (theta_vec): {theta_vec}")
print(f"Bias (offset0): {offset0:.4f}")
print(f"Epochs run: {epochs_ran}")
print(f"Train Accuracy: {train_acc:.3f}")
print(f"Test  Accuracy: {test_acc:.3f}")
print("Confusion Matrix (test):\n", cm)

plt.figure(figsize=(5,4))

# Scatter training points with new colors
plt.scatter(X_train_std[:,0],X_train_std[:,1],
            marker='o',label='train',alpha=0.8,c='dodgerblue')  # 🔹 New color
plt.scatter(X_test_std[:,0],X_test_std[:,1],
            marker='s',label='test',alpha=0.8,c='orange')       # 🔹 New color

x_vals = np.linspace(X_train_std[:,0].min()-0.5, X_train_std[:,0].max()+0.5,200)
if abs(theta_vec[1])>1e-8:
    y_vals = -(offset0 + theta_vec[0]*x_vals)/theta_vec[1]
    plt.plot(x_vals,y_vals,linewidth=2,label='decision boundary',c='crimson')  # 🔹 New boundary color
else:
    x_const = -offset0/(theta_vec[0]+1e-12)
    plt.axvline(x_const,linewidth=2,label='decision boundary',c='crimson')     # 🔹 New boundary color

plt.xlabel("grade_point (standardized)")
plt.ylabel("resume_rating (standardized)")
plt.title("Perceptron Boundary on Standardized Features",color='darkblue')     # 🔹 Title color
plt.legend(loc="best")
plt.grid(True, color='lightgray', linestyle='--', alpha=0.7)                     # 🔹 Grid style
plt.show()

plt.figure(figsize=(5,4))

# Scatter points with new colors
plt.scatter(X_train_std[:,0],X_train_std[:,1],
            marker='o',label='train',alpha=0.8,c='dodgerblue')      # 🔹 New color
plt.scatter(X_test_std[:,0],X_test_std[:,1],
            marker='s',label='test',alpha=0.8,c='goldenrod')        # 🔹 New color

# Common x-range
x_vals = np.linspace(X_train_std[:,0].min()-0.5, X_train_std[:,0].max()+0.5,200)

# Scratch boundary
if abs(theta_vec[1])>1e-8:
    y_scratch = -(offset0 + theta_vec[0]*x_vals)/theta_vec[1]
    plt.plot(x_vals,y_scratch,linewidth=2,label='scratch boundary',c='crimson')   # 🔹 Scratch boundary color
else:
    x_const = -offset0/(theta_vec[0]+1e-12)
    plt.axvline(x_const,linewidth=2,label='scratch boundary',c='crimson')

# Sklearn boundary
w = perc_sklearn.coef_.ravel()
b = float(perc_sklearn.intercept_)
if abs(w[1])>1e-8:
    y_sk = -(b + w[0]*x_vals)/w[1]
    plt.plot(x_vals,y_sk,linestyle='--',linewidth=2,label='sklearn boundary',c='limegreen')  # 🔹 Sklearn boundary color
else:
    x_const = -b/(w[0]+1e-12)
    plt.axvline(x_const,linestyle='--',linewidth=2,label='sklearn boundary',c='limegreen')

plt.xlabel("grade_point (standardized)")
plt.ylabel("resume_rating (standardized)")
plt.title("Perceptron Decision Boundaries: Scratch vs. Scikit-learn",color='midnightblue')  # 🔹 Title color
plt.legend(loc="best")
plt.grid(True,color='lightgray',linestyle='--',alpha=0.7)                                     # 🔹 Grid style
plt.show()

